In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import Sample_AlexNet
from SampleNN import *
from tqdm import tqdm_notebook
%matplotlib inline
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
import torchvision

In [2]:
import torchvision.transforms as transforms
traindir = '/home/yanzy/data/train'
valdir = '/home/yanzy/data/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_loader = torch.utils.data.DataLoader(
        torchvision.datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,

        ])),
        batch_size=128, shuffle=False,
        num_workers=4, pin_memory=True)

In [3]:
def val(Sample = 1, N = 16, m = 8):
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net, N, m)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                images = mSample(N=N,m=m)(images)
                outputs = sample_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    sample_net.load_state_dict(TMP)
    del TMP
    return 100.0 * correct / total

In [4]:
def val_float(Sample = 1, N = 16, m = 8):
    float_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = float_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)


                correct += (predicted == labels).sum().item()
                #print correct, total

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    return 100.0 * correct / total

## With Clamp version

In [5]:
float_net = torchvision.models.alexnet(pretrained=True).to(device)
gt  = float_net.state_dict()
b = gt.keys()
N = 16
m = 10

In [6]:
sample_net = Sample_AlexNet.alexnet().to(device)
lol = sample_net.state_dict()
a = lol.keys()
for i in range(len(a)):
    lol[a[i]] = gt[b[i]]
    print (lol[a[i]].shape)
sample_net.load_state_dict(lol)
sampleStateDict(sample_net, N, m)
sample_net.eval()
print (len(a))

torch.Size([64, 3, 11, 11])
torch.Size([64])
torch.Size([192, 64, 5, 5])
torch.Size([192])
torch.Size([384, 192, 3, 3])
torch.Size([384])
torch.Size([256, 384, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([4096, 9216])
torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096])
torch.Size([1000, 4096])
torch.Size([1000])
16


In [10]:
val(1,N,13)

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdtID0gMTMnLCBtYXg9NTAwMDAsIHN0eWxlPVByb2dyZXNzU3R5bGUoZGVzY3JpcHRpb25fd2lkdGg9dSfigKY=


When m = 13, Accuracy of the network on the 50000 test images: 44.048 %


44.048

In [11]:
val_float()

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdtID0gOCcsIG1heD01MDAwMCwgc3R5bGU9UHJvZ3Jlc3NTdHlsZShkZXNjcmlwdGlvbl93aWR0aD11J2nigKY=


When m = 8, Accuracy of the network on the 50000 test images: 47.992 %


47.992

In [8]:
import time
happysfsd = 0
with torch.no_grad():
    for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            images, labels = data
            if labels == 23:
                happysfsd = happysfsd + 1
            if happysfsd == 13:
                break

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdtID0gOCcsIG1heD01MDAwMCwgc3R5bGU9UHJvZ3Jlc3NTdHlsZShkZXNjcmlwdGlvbl93aWR0aD11J2nigKY=


In [8]:
print torch.max(sample_net(mSample(N=16,m=8)(images).to(device)).data, 1)
print labels

(tensor([14.3750], device='cuda:0'), tensor([138], device='cuda:0'))
tensor([23])


In [76]:
M = (images.to(torch.float32)/pow(2,-8)).round().clamp(-pow(2,16),pow(2,16)-1).to(torch.int32)


In [8]:
a = torch.load('./Weights/features.0.weight.pt')
print(a.size())

torch.Size([64, 3, 11, 11])


In [11]:
N = 16
m = 8
delt = pow(2,-m)
Q = pow(2, N-1) - 1
ImageSize = 224
img = torchvision.transforms.ToTensor()(plt.imread('timg.jpg'))
img = nn.functional.interpolate(img.view(1,3,650,1200),size=(ImageSize,ImageSize),mode='bilinear')
img = (img/delt).to(torch.int16)
img = img.to(torch.float) * delt

In [25]:
sample_net.eval()
a = open('test.csv', 'w+')
for i in list((mSample(N,m)(sample_net(img))/delt).view(-1).detach().numpy()):
    a.write('%d,'%i)

In [26]:
a.close()

In [55]:
sample_net(img).argmax()

tensor(803)

In [54]:
float_net.eval()
float_net(img).argmax()

tensor(803)

In [43]:
torchvision.transforms.ToTensor()(plt.imread('timg.jpg'))

tensor([[[0.7647, 0.7569, 0.7490,  ..., 0.6549, 0.6627, 0.6627],
         [0.7686, 0.7608, 0.7647,  ..., 0.6471, 0.6667, 0.6667],
         [0.7686, 0.7569, 0.7725,  ..., 0.6471, 0.6353, 0.6431],
         ...,
         [0.4078, 0.3804, 0.4157,  ..., 0.2706, 0.5373, 0.5882],
         [0.3922, 0.3843, 0.3843,  ..., 0.2157, 0.4471, 0.5843],
         [0.5059, 0.5176, 0.5137,  ..., 0.2078, 0.3529, 0.5255]],

        [[0.7294, 0.7255, 0.7176,  ..., 0.6549, 0.6627, 0.6627],
         [0.7333, 0.7255, 0.7294,  ..., 0.6392, 0.6588, 0.6588],
         [0.7412, 0.7294, 0.7451,  ..., 0.6392, 0.6275, 0.6353],
         ...,
         [0.2902, 0.2588, 0.2941,  ..., 0.2353, 0.4431, 0.5412],
         [0.2706, 0.2627, 0.2627,  ..., 0.2000, 0.3804, 0.5255],
         [0.4314, 0.4431, 0.4392,  ..., 0.1922, 0.2824, 0.4706]],

        [[0.6941, 0.6824, 0.6745,  ..., 0.6549, 0.6627, 0.6627],
         [0.6980, 0.6902, 0.6941,  ..., 0.6510, 0.6706, 0.6706],
         [0.7020, 0.6902, 0.7059,  ..., 0.6510, 0.6392, 0.